In [ ]:
%load_ext autoreload
%autoreload 2
%xmode Plain

Exception reporting mode: Plain


In [ ]:
import pandas as pd
import plotly
import plotly.express as px 
import dtale
import numpy as np
import random
from bazaraki import utils
from tqdm import tqdm
import swifter
from pathlib import Path
import datacompy
from datetime import datetime, date
from parse import parse
from glob import glob 


/tmp/ipykernel_1293838/3232598961.py:11: UserWarning:

SparkPandasCompare currently only supports Numpy < 2.Please note that the SparkPandasCompare functionality will not work and currently is not supported.



In [7]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Create a session with retry strategy
session = requests.Session()

# Add retry strategy
retry_strategy = Retry(
    total=3,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504],
)
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("http://", adapter)
session.mount("https://", adapter)

# More comprehensive headers that better mimic a real browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "sec-ch-ua": '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "Cache-Control": "max-age=0"
}

# Update session headers
session.headers.update(headers)

try:
    response = session.get("https://www.bazaraki.com/real-estate-to-rent/", timeout=30)
    print(f"Status Code: {response.status_code}")
    
    if response.status_code == 200:
        print("Success! First 1000 characters:")
        print(response.text[:1000])
    else:
        print("Response headers:")
        for key, value in response.headers.items():
            print(f"{key}: {value}")
        print("\nFirst 1000 characters of response:")
        print(response.text[:1000])
        
except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")

cloudscraper not available, trying with requests and custom SSL context
Request failed: HTTPSConnectionPool(host='www.bazaraki.com', port=443): Max retries exceeded with url: /real-estate-to-rent/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)')))


In [2]:
headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-language": "en-US,en;q=0.9",
    "priority": "u=0, i",
    "sec-ch-ua": "\"Not)A;Brand\";v=\"8\", \"Chromium\";v=\"138\", \"Google Chrome\";v=\"138\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"macOS\"",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1"
  }

In [3]:
import requests
# headers = {
#     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
#     "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
#     "Accept-Language": "en-US,en;q=0.5",
#     "Connection": "keep-alive"
# }
r = requests.get("https://www.bazaraki.com/real-estate-to-rent/", headers=headers)
print(r.status_code, r.text)

403 <!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,"Helvetica Neue",Arial,"Noto Sans",sans-serif,"Apple Color Emoji","Segoe UI Emoji","Segoe UI Symbol","Noto Color Emoji"}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;padding-left:1.5rem;max-width:60rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{line-height:2.25rem;font-size:1.5rem;font-weight:500}@media (width <= 720px){.h2{line-height:1.5rem;font-size:1.25rem}}#challenge-error-text{background-image:url("");background-repeat:no

In [4]:
pd.set_option('display.max_rows', 50)  # Disable row limit
pd.set_option('display.max_columns', 40)  # Disable column limit
pd.set_option('display.width', 20)  # Disable line width limit
pd.set_option('display.max_colwidth', 100)  # Disable column width limit
# Set the display precision to 2 decimal places  
pd.set_option('display.precision', 2)  
  
# Or set the float format to always show 2 decimal places  
pd.set_option('display.float_format', '{:.2f}'.format)  


In [5]:
pd.options.plotting.backend = "plotly"
plotly.io.renderers.default = "notebook_connected"


In [37]:
df = utils.read_dfs("output/*.parquet")

Reading output/2024-12-12 18:34:25 real-estate-to-rent_real-estate-for-sale.parquet
Reading output/2024-12-14 11:44:22 real-estate-to-rent_real-estate-for-sale.parquet
Total: 35559 read: 34669 new: 745 deleted: 890
Reading output/2024-12-15 18:00:14 real-estate-to-rent_real-estate-for-sale.parquet
Total: 35734 read: 34398 new: 175 deleted: 490
Reading output/2024-12-16 23:13:52 real-estate-to-rent_real-estate-for-sale.parquet


Total: 36281 read: 34510 new: 547 deleted: 647
Reading output/2024-12-17 21:31:02 real-estate-to-rent_real-estate-for-sale.parquet
Total: 37085 read: 34681 new: 804 deleted: 800
Reading output/2024-12-18 23:01:10 real-estate-to-rent_real-estate-for-sale.parquet
Total: 37577 read: 34809 new: 492 deleted: 464
Reading output/2024-12-19 22:09:26 real-estate-to-rent_real-estate-for-sale.parquet
Total: 38047 read: 33723 new: 470 deleted: 1661
Reading output/2024-12-22 21:49:27 real-estate-to-rent_real-estate-for-sale.parquet
Total: 39161 read: 33255 new: 1114 deleted: 1833
Reading output/2024-12-23 19:08:45 real-estate-to-rent_real-estate-for-sale.parquet
Total: 39512 read: 33936 new: 351 deleted: 439
Reading output/2024-12-27 17:28:17 real-estate-to-rent_real-estate-for-sale.parquet
Total: 39938 read: 32909 new: 426 deleted: 1605
Reading output/2024-12-29 13:00:58 real-estate-to-rent_real-estate-for-sale.parquet
Total: 40131 read: 32452 new: 193 deleted: 756
Reading output/2025-01-02 12:05:

In [ ]:
df = utils.add_city_disctrict_cols(df)
df.iloc[0].T

url                                                 https://www.bazaraki.com/adv/5415213_4-bedroom-detached-house-to-rent/
title                                                                                     4-bedroom detached house to rent
price                                                                                                              1650.00
original_price                                                                                                         NaN
price_per_sqm                                                                                                          NaN
location                                                                                               Larnaca, Dromolaxia
posted                                                                                                     Yesterday 20:57
reference_number                                                                                                       NaN
views           

In [40]:
# remove invalid properties
df = utils.filter_in(df, "`Property area` > 10 and `Property area` < 300")

removing 20873/85606 rows


In [41]:
df.price_per_sqm = df.price / (df["Property area"]).round(2)

In [42]:
# selected_df = saledf[saledf.price < 5000]
# print(f"removed {len(saledf) - len(selected_df)} rows")
# df = selected_df

In [43]:
# selected_df = df.query("`Bedrooms` in ('3', '2', '1', 'Studio')")
# print(f"removed {len(df) - len(selected_df)} rows")
# df = selected_df

In [44]:
def stats(saledf, rentdf):
    salestatdf = saledf.query("Condition == 'Resale'").groupby(["city", "Bedrooms"]).price_per_sqm.describe()
    rentstatdf = rentdf.groupby(["city", "Bedrooms"]).price_per_sqm.describe()
    common_index = salestatdf.index.intersection(rentstatdf.index)
    apr = rentstatdf.loc[common_index] / salestatdf.loc[common_index] * 12 * 100
    return pd.concat([apr, rentstatdf, salestatdf], axis=1, keys=['apr', 'rent', 'sale']).sort_values(by=('apr', '50%'), ascending=False).dropna()
    
print("apparts")
display(stats(df.query("cat1 == 'Apartments, flats for sale'"), df.query("cat1 == 'Apartments, flats to rent'")))
print("houses")
display(stats(df.query("cat1 == 'Houses for sale'"), df.query("cat1 == 'Houses to rent'")))

apparts


apr  \
                        count   
city      Bedrooms              
Famagusta 4            800.00   
Limassol  6 and more  1200.00   
Larnaca   Studio      1800.00   
Limassol  5           1300.00   
Paphos    Studio      4357.89   
Larnaca   4            651.43   
          1           1776.34   
Famagusta Studio      1636.36   
Larnaca   3            983.44   
Limassol  Studio     18240.00   
Larnaca   2           1933.26   
Nicosia   Studio      4954.84   
Limassol  1           6098.51   
Nicosia   1           8479.59   
Paphos    1           2087.23   
Nicosia   3           2201.23   
Paphos    2           2059.39   
Nicosia   2           3859.28   
Limassol  2           3747.51   
Nicosia   4           1554.55   
Paphos    3           1597.55   
Famagusta 2            611.76   
          3            331.91   
Limassol  3           2779.36   
          4           1354.29   
Famagusta 1            658.54   
Paphos    4           3120.00   

                            \
                      mean   
city      Bedrooms           
Famagusta 4          22.44   
Limassol  6 and more 28.24   
Larnaca   Studio      8.94   
Limassol  5           6.71   
Paphos    Studio      8.31   
Larnaca   4           6.16   
          1           6.66   
Famagusta Studio      3.50   
Larnaca   3           6.30   
Limassol  Studio      7.22   
Larnaca   2           6.65   
Nicosia   Studio      6.97   
Limassol  1           5.29   
Nicosia   1           6.14   
Paphos    1           6.04   
Nicosia   3           5.80   
Paphos    2           5.96   
Nicosia   2           5.94   
Limassol  2           5.58   
Nicosia   4           4.60   
Paphos    3          21.08   
Famagusta 2           4.50   
          3           4.72   
Limassol  3           4.81   
          4           4.16   
Famagusta 1           4.31   
Paphos    4           4.51   

                             \
                        std   
city      Bedrooms            
Famagusta 4            6.74   
Limassol  6 and more 118.12   
Larnaca   Studio       6.39   
Limassol  5            3.84   
Paphos    Studio       8.23   
Larnaca   4            3.53   
          1            2.92   
Famagusta Studio       0.42   
Larnaca   3            3.94   
Limassol  Studio       9.61   
Larnaca   2           53.96   
Nicosia   Studio       7.54   
Limassol  1            3.59   
Nicosia   1            7.26   
Paphos    1            5.14   
Nicosia   3            4.42   
Paphos    2            5.67   
Nicosia   2            5.31   
Limassol  2           38.66   
Nicosia   4            2.81   
Paphos    3          318.75   
Famagusta 2            2.37   
          3            3.56   
Limassol  3            4.30   
          4            3.77   
Famagusta 1            4.31   
Paphos    4            4.42   

                             \
                        min   
city      Bedrooms            
Famagusta 4           37.08   
Limassol  6 and more   2.96   
Larnaca   Studio       7.32   
Limassol  5            8.14   
Paphos    Studio      18.64   
Larnaca   4            6.80   
          1            8.43   
Famagusta Studio      11.41   
Larnaca   3            6.32   
Limassol  Studio       3.61   
Larnaca   2            5.59   
Nicosia   Studio      14.20   
Limassol  1            1.56   
Nicosia   1          237.50   
Paphos    1            6.89   
Nicosia   3          384.00   
Paphos    2           16.60   
Nicosia   2            6.59   
Limassol  2            1.67   
Nicosia   4            3.32   
Paphos    3            0.72   
Famagusta 2            7.45   
          3            5.36   
Limassol  3            4.41   
          4            5.53   
Famagusta 1            7.48   
Paphos    4            5.76   

                            \
                       25%   
city      Bedrooms           
Famagusta 4          27.33   
Limassol  6 and more  3.30   
Larnaca   Studio     10.05   
Limassol  5           7.21   
Paphos    Studio      8.06   
Larnaca   4           6.97   
  

houses


apr  \
                     count   
city      Bedrooms           
Limassol  1        8220.00   
Nicosia   Studio   1800.00   
Larnaca   4         523.33   
Nicosia   5         384.00   
Paphos    1        2123.08   
Larnaca   2        1151.72   
          5         498.70   
Nicosia   2        1609.09   
Larnaca   3         723.91   
Limassol  2        2568.65   
          6         876.92   
Paphos    2        1239.67   
Nicosia   4         606.94   
Paphos    5         612.24   
Nicosia   3         942.86   
Limassol  3        1814.07   
Paphos    4        1120.00   
Limassol  4        1146.84   
          5         830.77   
Nicosia   1        2880.00   
Paphos    3         917.65   
Larnaca   1        1107.69   
Paphos    6         450.00   
Famagusta 2         533.33   
          3         404.04   
          4         290.53   
          5          92.31   

                          \
                    mean   
city      Bedrooms         
Limassol  1         9.04   
Nicosia   Studio   12.86   
Larnaca   4         5.58   
Nicosia   5         6.54   
Paphos    1         6.52   
Larnaca   2         6.13   
          5         5.52   
Nicosia   2         5.44   
Larnaca   3         5.87   
Limassol  2         5.64   
          6         6.21   
Paphos    2         5.46   
Nicosia   4         5.00   
Paphos    5         4.35   
Nicosia   3         5.26   
Limassol  3         5.07   
Paphos    4         4.34   
Limassol  4         4.73   
          5         5.11   
Nicosia   1         5.12   
Paphos    3         4.86   
Larnaca   1         5.02   
Paphos    6         4.30   
Famagusta 2         4.15   
          3         4.45   
          4         4.83   
          5         3.37   

                          \
                     std   
city      Bedrooms         
Limassol  1         6.74   
Nicosia   Studio   18.87   
Larnaca   4         2.82   
Nicosia   5         5.65   
Paphos    1         4.14   
Larnaca   2        10.20   
          5         3.21   
Nicosia   2         3.59   
Larnaca   3         5.82   
Limassol  2         4.61   
          6         7.93   
Paphos    2         5.55   
Nicosia   4         2.38   
Paphos    5         3.71   
Nicosia   3         5.13   
Limassol  3         6.48   
Paphos    4         1.52   
Limassol  4         4.15   
          5         5.09   
Nicosia   1         6.96   
Paphos    3         4.62   
Larnaca   1         4.90   
Paphos    6         2.06   
Famagusta 2         3.36   
          3         8.82   
          4        10.70   
          5         0.98   

                            \
                       min   
city      Bedrooms           
Limassol  1           8.82   
Nicosia   Studio     11.25   
Larnaca   4          13.20   
Nicosia   5          11.75   
Paphos    1           7.62   
Larnaca   2           6.96   
          5           9.74   
Nicosia   2           8.68   
Larnaca   3        1152.00   
Limassol  2           6.40   
          6          10.43   
Paphos    2           6.00   
Nicosia   4           3.95   
Paphos    5          13.50   
Nicosia   3           9.12   
Limassol  3          13.20   
Paphos    4           2.75   
Limassol  4          11.33   
          5           7.64   
Nicosia   1           4.18   
Paphos    3           3.17   
Larnaca   1           8.12   
Paphos    6           9.83   
Famagusta 2           3.00   
          3           5.76   
          4           6.84   
          5          10.01   

                          \
                     25%   
city      Bedrooms         
Limassol  1        12.35   
Nicosia   Studio    9.93   
Larnaca   4         6.75   
Nicosia   5         6.00   
Paphos    1         8.56   
Larnaca   2         7.31   
          5         6.66   
Nicosia   2         6.05   
Larnaca   3         5.96   
Limassol  2         6.20   
          6         7.92   
Paphos    2         5.23   
Nicosia   4         5.13   
Paphos    5         5.80   
Nicosia   3         5.30   
Limassol  3         5.34   
Paphos 

In [45]:
rentdf = df.query("cat1 == 'Apartments, flats for sale' and price_per_sqm < 6000")
px.histogram(rentdf, x="price_per_sqm", barmode='overlay', facet_col="city", color="Bedrooms", title="Apts for sale per city/bedrooms").show()


In [46]:
rentdf = df.query("cat1 == 'Apartments, flats for sale' and price < 1000000")
px.histogram(rentdf, x="price", barmode='overlay', facet_col="city", color="Bedrooms", title="Apts sale prices per city/bedrooms").show()


In [47]:
rentdf = df.query("cat1 == 'Apartments, flats to rent' and price < 3000")
px.histogram(rentdf, x="price", barmode='overlay', facet_col="city", color="Bedrooms", title="Apts rent prices per city/bedrooms").show()


In [48]:
df.sort_values("views", ascending=False)


,url,title,price,original_price,price_per_sqm,location,posted,reference_number,views,lat,lng,sold,cat0,cat1,Property area,Pets,Type,Parking,Plot area,Furnishing,...,Construction year,Reference number,Condition,Square meter price,Minimum stay,Land type,Plot Type,Parcel number,Planning zone,Registration number,Share,Density,Coverage,Registration block,Area,Pick a point,posted_dt,delete_date,city,district
ad_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4025924,https://www.bazaraki.com/adv/4025924_1-bedroom-apartment-to-rent/,Studio apartment to rent,600.00,NaN,17.14,"Limassol, Trimiklini",18.11.2024 13:34,NaN,27310,34.85,32.91,False,Cyprus real estate to rent,"Apartments, flats to rent",35.00,Not allowed,Apartment,Covered,NaN,Fully Furnished,...,2023,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,NaN,None,2024-11-18T13:34:00,NaN,Limassol,Trimiklini
1241192,https://www.bazaraki.com/adv/1241192_serviced-offices-for-rent/,Serviced offices in limassol,650.00,NaN,59.09,"Limassol, Limassol - Agia Zoni",05.12.2024 15:10,NaN,22598,34.69,33.04,False,Cyprus real estate to rent,Commercial property,11.00,None,Offices,None,NaN,None,...,None,None,None,€59 /,None,None,None,None,None,NaN,None,None,None,NaN,NaN,None,2024-12-05T15:10:00,2025-02-10,Limassol,Limassol - Agia Zoni
4095853,https://www.bazaraki.com/adv/4095853_1-bedroom-detached-house-for-sale/,2-bedroom detached house fоr sаle,70000.00,NaN,1272.73,"Paphos, Filousa Kelokedaron",29.11.2024 18:41,NaN,18964,NaN,NaN,False,Cyprus real estate for sale,Houses for sale,55.00,None,Detached house,None,156.00,Fully Furnished,...,None,None,Resale,€1.273 /,None,None,None,None,None,NaN,None,None,None,NaN,NaN,None,2024-11-29T18:41:00,2025-01-02,Paphos,Filousa Kelokedaron
2291746,https://www.bazaraki.com/adv/2291746_2-preserved-houses-2-diateretees-oikies/,3-bedroom detached house fоr sаle,75000.00,80000.00,294.12,"Nicosia, Moni Kato",12.11.2024 07:25,NaN,18630,35.07,33.10,False,Cyprus real estate for sale,Houses for sale,255.00,None,Detached house,No,NaN,Unfurnished,...,None,None,Resale,€294 /,None,None,None,None,None,NaN,None,None,None,NaN,NaN,None,2024-11-12T07:25:00,2024-12-14,Nicosia,Moni Kato
4256623,https://www.bazaraki.com/adv/4256623_2-bedroom-detached-house-to-rent/,2-bedroom detached house to rent,700.00,NaN,5.83,"Limassol, Kaminaria",02.12.2024 06:19,NaN,16077,NaN,NaN,False,Cyprus real estate to rent,Houses to rent,120.00,Not allowed,Detached house,No,NaN,Fully Furnished,...,2012,None,None,None,None,None,None,None,None,NaN,None,None,None,NaN,NaN,None,2024-12-02T06:19:00,2025-01-02,Limassol,Kaminaria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5747292,https://www.bazaraki.com/adv/5747292_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,340000.00,NaN,4533.33,"Paphos, Chlorakas",01:17,NaN,0,34.81,32.40,False,Cyprus real estate for sale,"Apartments, flats for sale",75.00,None,Apartment,Covered,NaN,None,...,None,3825,Under construction,€4.533 /,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,2025-04-11T01:17:00,NaN,Paphos,Chlorakas
5747293,https://www.bazaraki.com/adv/5747293_2-bedroom-apartment-for-sale/,2-bedroom apartment fоr sаle,340000.00,NaN,4533.33,"Paphos, Chlorakas",01:17,NaN,0,34.81,32.40,False,Cyprus real estate for sale,"Apartments, flats for sale",75.00,None,Apartment,Covered,NaN,None,...,None,3826,Under construction,€4.533 /,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,2025-04-11T01:17:00,NaN,Paphos,Chlorakas
5746404,https://www.bazaraki.com/adv/5746404_3-bedroom-apartment-for-sale/,3-bedroom apartment fоr sаle,890000.00,NaN,8317.76,"Limassol, Historical Center",Yesterday 16:24,NaN,0,34.68,33.05,False,Cyprus real estate for sale,"Apartments, flats for sale",107.00,None,Apartment,Covered,NaN,None,...,None,55208,Under construction,€8.318 /,None,None,None,None,None,NaN,None,None,None,NaN,NaN,NaN,2025-04-10T16:24:00,NaN,

In [49]:
df = pd.read_parquet("/home/ubuntu/bazaraki/output/2025-04-25 00:00:00 real-estate-to-rent_real-estate-for-sale.parquet")
df

,url,title,price,original_price,price_per_sqm,location,posted,ad_id,reference_number,views,lat,lng,sold,posted_dt,cat0,cat1,Reference number,Property area,Type,Floor,...,Bedrooms,Bathrooms,images,description,Pets,Postal code,Plot area,Square meter price,Minimum stay,Share,Land type,Plot Type,Parcel number,Density,Coverage,Planning zone,Registration number,Area,Condition,Registration block
0,https://www.bazaraki.com/adv/5764421_3-bedroom-apartment-to-rent/,3-bedroom apartment to rent,1000,NaN,NaN,"Larnaca, Larnaka - Agios Nikolaos",23.04.2025 15:39,5764421,NaN,20,34.93,33.60,False,2025-04-23T15:39:00,Cyprus real estate to rent,"Apartments, flats to rent",40597,85.00,Apartment,2nd,...,3,1,"[https://cdn1.bazaraki.com/media/cache1/0b/2d/0b2d1c375b300fa8261c1667d3a185a2.webp, https://cdn...","Cozy Three Bedroom Apartment for Rent in New Mall Area, Larnaca. Within close proximity to the N...",None,NaN,NaN,None,None,None,None,None,None,None,None,None,NaN,NaN,None,NaN
1,https://www.bazaraki.com/adv/5764602_3-bedroom-apartment-to-rent/,3-bedroom apartment to rent,2900,NaN,NaN,"Limassol, Limassol - Agia Zoni",23.04.2025 16:37,5764602,NaN,18,34.69,33.05,False,2025-04-23T16:37:00,Cyprus real estate to rent,"Apartments, flats to rent",1547403,120.00,Apartment,3rd,...,3,2,"[https://cdn1.bazaraki.com/media/cache1/e2/9c/e29c2e8ca9c5333fa9e5796ae8dfdcfc.webp, https://cdn...","Apartment, floor: 3rd, in Limassol City - Agia Zoni. The Apartment is 120 sq.m.. It consists of:...",None,NaN,NaN,None,None,None,None,None,None,None,None,None,NaN,NaN,None,NaN
2,https://www.bazaraki.com/adv/5764825_3-bedroom-apartment-to-rent/,3-bedroom apartment to rent,2100,NaN,NaN,"Limassol, Limassol - Omonia",23.04.2025 18:47,5764825,NaN,28,34.67,33.01,False,2025-04-23T18:47:00,Cyprus real estate to rent,"Apartments, flats to rent",None,111.00,Apartment,3rd,...,3,3,"[https://cdn1.bazaraki.com/media/cache1/0b/e7/0be7ccfbce36571e1dacc75767fb9d6c.webp, https://cdn...",✨Fully Renovated 3-Bedroom Apartment – Prime Central Location! 🏡🌆\nDiscover exceptional city liv...,Allowed,NaN,NaN,None,None,None,None,None,None,None,None,None,NaN,NaN,None,NaN
3,https://www.bazaraki.com/adv/5763152_2-bedroom-apartment-to-rent/,2-bedroom apartment to rent,800,NaN,NaN,"Nicosia, Agioi Omologites",22.04.2025 19:02,5763152,NaN,15,NaN,NaN,False,2025-04-22T19:02:00,Cyprus real estate to rent,"Apartments, flats to rent",64230,82.00,Apartment,3rd,...,2,1,"[https://cdn1.bazaraki.com/media/cache1/27/05/27054f40e6e5232ac84943e6fc89fc6d.webp, https://cdn...",Partially Renovated Apartment for Rent in Agioi Omologites - just steps away from KPMG!\nThis a...,None,NaN,NaN,None,None,None,None,None,None,None,None,None,NaN,NaN,None,NaN
4,https://www.bazaraki.com/adv/5764816_2-bedroom-apartment-to-rent/,2-bedroom apartment to rent,1800,NaN,NaN,"Limassol, Limassol - Neapolis",23.04.2025 18:48,5764816,NaN,52,34.69,33.06,False,2025-04-23T18:48:00,Cyprus real estate to rent,"Apartments, flats to rent",RR16236,75.00,Apartment,2nd,...,2,1,"[https://cdn1.bazaraki.com/media/cache1/09/1f/091ff6a151014a2f23dc2eb843d53eb8.webp, https://cdn...",A new condition two (2) bedroom apartment developed in 2012 is now available for immediate rent ...,None,3101.00,NaN,None,None,None,None,None,None,None,None,None,NaN,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38004,https://www.bazaraki.com/adv/5692055_2-bedroom-apartment-to-rent/,2-bedroom apartment to rent,1650,NaN,NaN,"Limassol, Polemidia Kato",23.04.2025 16:04,5692055,NaN,1975,34.69,33.01,False,2025-04-23T16:04:00,Cyprus real estate to rent,"Apartments, flats to rent",None,110.00,Apartment,3rd,...,2,2,"[https://cdn1.bazaraki.com/media/cache1/98/4b/984bc92a89162a0ccb3987f7a7fd717e.webp, https://cdn...",Discover these stylish key-ready apartments in a newly built apartment building in Kato Polemidi...,Allowed,NaN,NaN,None,None,None,None,None,No

In [50]:
df.price_per_sqm = df.price / (df["Property area"]).round(2)
df = utils.add_city_disctrict_cols(df).\
    query("cat1 == 'Apartments, flats for sale' and price_per_sqm < 20000 and Bedrooms in ('2', '1', 'Studio')").\
    sort_values(["Bedrooms", "Condition"])
df

,url,title,price,original_price,price_per_sqm,location,posted,ad_id,reference_number,views,lat,lng,sold,posted_dt,cat0,cat1,Reference number,Property area,Type,Floor,...,images,description,Pets,Postal code,Plot area,Square meter price,Minimum stay,Share,Land type,Plot Type,Parcel number,Density,Coverage,Planning zone,Registration number,Area,Condition,Registration block,city,district
7093,https://www.bazaraki.com/adv/5393097_2-bedroom-penthouse-for-sale/,1-bedroom penthouse fоr sаle,303000,495000.00,3835.44,"Limassol, Germasogeia",16.04.2025 09:24,5393097,NaN,436,34.71,33.08,False,2025-04-16T09:24:00,Cyprus real estate for sale,"Apartments, flats for sale",SORA303,79.00,Penthouse,3rd,...,"[https://cdn1.bazaraki.com/media/cache1/09/54/095444647882ddaa4207e00ae95f3b89.webp, https://cdn...","Last Available Apartment – Penthouse in Columbia, Limassol\nOnly one penthouse remains in this e...",None,4043.00,NaN,€3.835 /,None,None,None,None,None,None,None,None,NaN,NaN,Brand new,NaN,Limassol,Germasogeia
7102,https://www.bazaraki.com/adv/5755871_1-bedroom-apartment-for-sale/,1-bedroom apartment fоr sаle,145000,NaN,2589.29,"Nicosia, Lakatameia - Agios Nikolaos",16.04.2025 13:45,5755871,NaN,165,35.12,33.31,False,2025-04-16T13:45:00,Cyprus real estate for sale,"Apartments, flats for sale",None,56.00,Apartment,1st,...,"[https://cdn1.bazaraki.com/media/cache1/22/00/2200de3d25bd968632cc73e2f5ee1062.webp, https://cdn...",*ΛΑΚΑΤΑΜΙΑ\n*ΠΩΛΕΙΤΑΙ ΔΙΑΜ 1 ΥΠΝ\n*ΚΑΙΝΟΥΡΓΙΟ ΕΠΙΠΛΩΜΕΝO\n*€145.000+ΦΠΑ\n*Απεναντι απο γραμμικο ...,None,NaN,NaN,€2.589 /,None,None,None,None,None,None,None,None,NaN,NaN,Brand new,NaN,Nicosia,Lakatameia - Agios Nikolaos
7142,https://www.bazaraki.com/adv/5196281_1-bedroom-apartment-for-sale/,1-bedroom apartment fоr sаle,135000,NaN,2596.15,"Nicosia, Aglantzia",21.04.2025 17:00,5196281,NaN,102,35.16,33.41,False,2025-04-21T17:00:00,Cyprus real estate for sale,"Apartments, flats for sale",None,52.00,Apartment,1st,...,"[https://cdn1.bazaraki.com/media/cache1/74/9c/749cc38d81da9887f6d00751f1f59869.webp, https://cdn...","Διαμερίσματα 1 υπνοδωματίου , 900 μ. από το Πανεπιστήμιο Κύπρου στην Αγλαντζιά!\n📣Mονάρι 52 τ.μ...",None,2032.00,NaN,€2.596 /,None,None,None,None,None,None,None,None,NaN,NaN,Brand new,NaN,Nicosia,Aglantzia
7270,https://www.bazaraki.com/adv/5721010_1-bedroom-apartment-for-sale/,1-bedroom apartment fоr sаle,209000,NaN,3073.53,"Limassol, Polemidia Kato",00:11,5721010,NaN,0,NaN,NaN,False,2025-04-25T00:11:00,Cyprus real estate for sale,"Apartments, flats for sale",SA32104,68.00,Apartment,2nd,...,"[https://cdn1.bazaraki.com/media/cache1/ba/c8/bac8aeb127a4d7af540e46e0e0ae15e5.webp, https://cdn...","Reference ID: #SA32104Price details: €209,000 +VATProject occupy two three-storey buildings and ...",None,NaN,NaN,€3.074 /,None,None,None,None,None,None,None,None,NaN,NaN,Brand new,NaN,Limassol,Polemidia Kato
7290,https://www.bazaraki.com/adv/5625260_1-bedroom-apartment-for-sale/,1-bedroom apartment fоr sаle,160000,NaN,2352.94,"Larnaca, Aradippou - Apostolos Loukas",00:13,5625260,NaN,0,NaN,NaN,False,2025-04-25T00:13:00,Cyprus real estate for sale,"Apartments, flats for sale",120074,68.00,Apartment,None,...,"[https://cdn1.bazaraki.com/media/cache1/6f/23/6f233e07cc66b83d0aa09fc14d583204.webp, https://cdn...","New apartment for sale in a modern complex in Larnaca, Araddipou. The apartment has a large livi...",None,4102.00,NaN,€2.353 /,None,None,None,None,None,None,None,None,NaN,NaN,Brand new,NaN,Larnaca,Aradippou - Apostolos Loukas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35408,https://www.bazaraki.com/adv/5610856_studio-apartment-for-sale/,Studio apartment fоr sаle,105000,NaN,3000.00,"Nicosia, Aglantzia",02.04.2025 10:45,5610856,NaN,422,35.16,33.40,False,2025-04-02T10:45:00,Cyprus real estate for sale,"Apartments, flats for sale",FC244 - 101,35.00,Apartment,None,...,"[https://cdn1.bazaraki.com/media/cache1/c

In [53]:
px.box(df, x=df["Bedrooms"] + "|" + df["Condition"], y="price_per_sqm", title="Prices per bedroom/condition", facet_col="city", height=1600)